# Covid19 New Delhi Dashboard


In [ ]:
import os

import psycopg2
import tweepy
import configparser
from google.cloud import vision
import datetime
import re
from PIL import Image
import requests
import io
import json

In [ ]:
# Initialising the keys required

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key = config.get('KEY', 'consumer_key')
consumer_secret = config.get('KEY', 'consumer_secret')
access_token = config.get('KEY', 'access_token')
access_token_secret = config.get('KEY', 'access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.get('KEY', 'json_loc')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
date = datetime.datetime.now()
file = "{}-centers.json".format(date.strftime("%d-%m-%Y"))
file_name = "{}-data.json".format(date.strftime("%d-%m-%Y"))

In [ ]:
# Fetching the latest Health Bulletin tweet from Chief Minister's Account

search_words = "Delhi Health Bulletin (from:CMODelhi)"

tweets = tweepy.Cursor(api.search,
                       q=search_words,
                       lang="en", ).items(1)

image_url = ""
# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    print(image_url)


In [ ]:
data_dict = dict()
y = datetime.datetime.now()
y -= datetime.timedelta(days=2)
data_dict['day'] = y.day
data_dict['month'] = y.month
data_dict['year'] = y.year


In [ ]:
# Converting Image to Text

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_url

response = client.text_detection(image=image)

out_string = ""
for text in response.text_annotations:
    string = text.description
    break

print(out_string)

if response.error.message:
    raise Exception(
        '{}\nFor more info on error messages, check: '
        'https://cloud.google.com/apis/design/errors'.format(
            response.error.message))

In [ ]:
# Extracting Active Cases
match = (re.search("Active Cases", out_string))
i = int(match.end())
match = (re.search("Total Number of Containment Zones as on date", out_string))
j = int(match.start())
data_dict['active_cases'] = out_string[i + 1:j - 1]

# Extracting Containment
match = (re.search("Total Number of Containment Zones as on date: ", out_string))
i = int(match.end())
match = (re.search("Calls received", out_string))
j = int(match.start())
data_dict['zones'] = out_string[i:j - 1]

In [ ]:
def getNum(str):
    if len(str) == 0: return
    s = ""
    for char in str:
        if char.isdigit():
            s += char
    if len(s) == 0: return
    return int(s)

In [ ]:
# Cropping the image to get more parameters

im = Image.open(requests.get(image_url, stream=True).raw)

# Setting the points for cropped image
left = 500
top = 127
right = 700
bottom = 300

# Cropped image of above dimension
im1 = im.crop((left, top, right, bottom))

img_byte_arr = io.BytesIO()
im1.save(img_byte_arr, format='PNG')
img_byte_arr = img_byte_arr.getvalue()

image = vision.Image(content=img_byte_arr)

client = vision.ImageAnnotatorClient()
response = client.text_detection(image=image)
texts = response.text_annotations
string = ""
for text in texts:
    string = text.description
    break
print(string)

l = 1
for word in string.split("\n"):
    i = getNum(word)
    if i == 24: continue
    if l == 1: data_dict["positive"] = i
    if l == 2: data_dict["tests"] = i
    if l == 4: data_dict["recovered"] = i
    if l == 5: data_dict["deaths"] = i
    l += 1

if response.error.message:
    raise Exception(
        '{}\nFor more info on error messages, check: '
        'https://cloud.google.com/apis/design/errors'.format(
            response.error.message))


In [ ]:
im = Image.open(requests.get(image_url, stream=True).raw)

# Setting the points for cropped image
left = 525
top = 590
right = 700
bottom = 750

# Cropped image of above dimension
im1 = im.crop((left, top, right, bottom))

img_byte_arr = io.BytesIO()
im1.save(img_byte_arr, format='PNG')
img_byte_arr = img_byte_arr.getvalue()

image = vision.Image(content=img_byte_arr)

client = vision.ImageAnnotatorClient()
response = client.text_detection(image=image)
texts = response.text_annotations

string = ""
for text in texts:
    string = text.description
    break
print(string)

l = 1
for word in string.split("\n"):
    i = getNum(word)
    if i == 24: continue
    if l == 1: data_dict["vaccinated"] = i
    if l == 2: data_dict["first_dose"] = i
    if l == 3: data_dict["second_dose"] = i
    if l >= 4: break
    l += 1

if response.error.message:
    raise Exception(
        '{}\nFor more info on error messages, check: '
        'https://cloud.google.com/apis/design/errors'.format(
            response.error.message))



In [ ]:
# Writing the data dictionary to JSON File

print(data_dict)
with open(file_name, "w") as outfile:
    json.dump(data_dict, outfile)

In [ ]:
def parseJSON(js, date, lis, dat):
    centres = js['centers']
    for center in centres:
        sessions = center['sessions']
        for ses in sessions:
            if (ses['date'] == date):
                val = dict()
                val['day'] = dat.day
                val['month'] = dat.month
                val['year'] = dat.year
                val['center_id'] = center['center_id']
                val['name'] = center['name']
                val['pincode'] = center['pincode']
                val['district_name'] = center['district_name']
                val['vaccine'] = ses['vaccine']
                val['min_age_limit'] = ses['min_age_limit']
                try:
                    if (center['fee_type'] == "Paid"):
                        val['fee'] = center['vaccine_fees'][0]['fee']
                    else:
                        val['fee'] = 0
                except:
                    val['fee'] = 0
                lis.append(val)

In [ ]:
# Fetching Vaccine Center Data
payload = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15'
}

i = 110001
j = 110096

lis = list()
url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={}&date={}"
y = datetime.datetime.now()
date = y.strftime("%d-%m-%Y")

while i <= j:
    response = requests.get(url.format(i, date), headers=headers, data=payload)
    parseJSON(response.json(), date, lis, y)
    i += 1


In [ ]:
# Writing the centers list to JSON File
with open(file, "w") as outfile:
    json.dump(lis, outfile)

In [ ]:
#Connecting to AWS Redshift
config = configparser.ConfigParser()
config.read('dwh.cfg')

# Connect to DB and return the connection
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()


In [ ]:
query = """
CREATE TABLE CENTERS
(
DAY INT,
MONTH INT,
YEAR INT,
CENTER_ID INT,
NAME VARCHAR,
DISTRICT_NAME VARCHAR,
PINCODE INT,
VACCINE VARCHAR,
MIN_AGE_LIMIT INT,
FEE INT
)"""

cur.execute(query)
conn.commit()


In [ ]:
query = """
CREATE TABLE DATA
(
DAY INT,
MONTH INT,
YEAR INT,
POSITIVE INT,
TESTS INT,
RECOVERED INT,
DEATHS INT,
VACCINATED INT,
FIRST_DOSE INT,
SECOND_DOSE INT,
ACTIVE_CASES INT,
CONT_ZONES INT
)"""

cur.execute(query)
conn.commit()

In [ ]:
# Adding data to DATA table
query = "INSERT INTO DATA " \
        "VALUES({day},{month},{year},{positive},{tests},{recovered},{deaths},{vaccinated},{first_dose},{second_dose},{active_cases},{zones})"

data = json.load(open(file_name))
str = query.format(**data)
print(str)
cur.execute(str)
conn.commit()


In [ ]:
# Adding data to CENTERS table
query = "INSERT INTO CENTERS(day,month,year,center_id,name,pincode,district_name,vaccine,min_age_limit,fee) " \
        "VALUES({day},{month},{year},{center_id},'{name}',{pincode},'{district_name}','{vaccine}',{min_age_limit},{fee})"

data = json.load(open(file))
for vaues in data:
    str = query.format(**vaues)
    print(str)
    cur.execute(str)
    conn.commit()


In [ ]:
# Closing the connection
conn.close()